In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import GlobalAveragePooling2D, BatchNormalization
from keras.layers.core import Dense, Flatten,  Dropout
from keras.layers.convolutional import Conv2D
import keras
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import Sequence
from keras.applications.mobilenet_v2 import MobileNetV2
from sklearn.model_selection import train_test_split
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import gzip
import os
import pickle
import cv2
import numpy as np
import pandas as pd
import glob

Using TensorFlow backend.


In [0]:
def collect_data(dirname, csvname):
    with open(filepath, 'ab') as f:
      data = []
      csv = pd.read_csv(csvname, sep=',')
      joy_values = csv['wheel'].values.tolist()
      images = glob.glob(dirname)
      count = 0
      for img in images:
          screenshot = cv2.imread(img)
          if count < len(joy_values):
              screenshot = transform_image(np.array(screenshot))
              data.append([screenshot, joy_values[count]])
          if count == len(images) - 1:
              print('Collected data count - {0}.'.format(count))
              pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
              data = [] 
          count += 1
def read_data(split=False):
    with gzip.open(filepath, 'rb') as f:
        data = []
        while True:
            try:
                temp = pickle.load(f)
                if type(temp) is not list:
                    temp = np.ndarray.tolist(temp)
                data = data + temp
            except EOFError:
                break
        if split:
            x_train = []
            y_train = []

            for i in range(0, len(data)):
                x_train.append(data[i][0])
                y_train.append(data[i][1])

            return np.array(x_train), np.array(y_train)
        else:
            return np.array(data)

def create_model():
    base_mobilenet_model = MobileNetV2(input_shape =  (100,400,3), 
                                    include_top = False, 
                                    weights = None,
                                     alpha=0.6)
    pilot_model = Sequential()
    pilot_model.add(BatchNormalization(input_shape =(100,400,3)))
    pilot_model.add(base_mobilenet_model)
    pilot_model.add(BatchNormalization())
    pilot_model.add(GlobalAveragePooling2D())
    pilot_model.add(Dropout(0.5))
    pilot_model.add(Dense(1, activation = 'linear' )) # linear is what 16bit did

    pilot_model.summary()

    pilot_model.compile(loss=keras.losses.mean_squared_error,
                  optimizer=keras.optimizers.Adam(lr=0.001),
                  metrics=[sign_pred])
    return pilot_model

def get_model():
    if os.path.isfile(modelpath):
        model = keras.models.load_model(modelpath)
    else:
        model = create_model()
    return model

def sign_pred(y_true, y_pred):
    mult = y_true * y_pred
    eq = keras.backend.cast(keras.backend.equal(keras.backend.sign(mult),keras.backend.ones_like(mult)), 'int32')
    return keras.backend.sum(eq) 

def train_model(model):
    x, y = read_data(True)
    x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.15, random_state=444)

    # test data set 0.15 : training data set 85%
    checkpoint = ModelCheckpoint('model-{epoch:03d}.h5', monitor='val_loss', verbose=1,save_best_only=True, mode='min')
    earlystop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=80)
    model.fit_generator(DataGenerator(x_train, y_train, 64, (100, 400), 3), steps_per_epoch= len(x_train)/64, epochs=200,
              validation_data=DataGenerator(x_valid, y_valid, 64, (100, 400), 3), validation_steps=len(y_valid)/64, callbacks=[checkpoint,earlystop], verbose=1)
    model.save(modelpath)

def display_data():
    data = read_data()
    data_size = len(data)
    print('Data size  -  ' + str(data_size))

    for i in range(data_size - 1, 0, -1):
        print(str(image[1]))
        cv2.waitKey(50)

def display(image, angle, label):
    plt.imshow(image)
    plt.xlabel("Steering angle: {:.10f}".format(angle))
    plt.title(label)

    plt.xticks([])
    plt.yticks([])
    plt.show()

class DataGenerator(Sequence):
    def __init__(self, X, y, batch_size, dim, n_channels, shuffle = True):
        self.X = X
        self.y = y if y is not None else y
        self.batch_size = batch_size
        self.dim = dim
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.X))
        if self.shuffle:
            np.random.shuffle(self.indexes)
            
    def __len__(self):
        return int(np.floor(len(self.X) / self.batch_size))
    
    def __data_generation(self, X_list, y_list):
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size))
        range_x = 50
        if y is not None:
            for i, (image, wheel) in enumerate(zip(X_list, y_list)):
                image = image / 255.0
                if np.random.rand() < 0.5:
                    image = cv2.flip(image, 1)
                    wheel = -wheel
                if np.random.rand() < 0.5:
                    trans_x = range_x * (np.random.rand() - 0.5)
                    wheel += trans_x * 0.01
                    trans_m = np.float32([[1, 0, trans_x], [0, 1, 0]])
                    height, width = image.shape[:2]
                    image = cv2.warpAffine(image, trans_m, (width, height))

                X[i] = image
                y[i] = wheel
            return X, y
        
        else:
            for i, img in enumerate(X_list):
                X[i] = img
                
            return X
        
    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
        X_list = [self.X[k] for k in indexes]
        
        if self.y is not None:
            y_list = [self.y[k] for k in indexes]
            X, y = self.__data_generation(X_list, y_list)
            return X, y
        else:
            y_list = None
            X = self.__data_generation(X_list, y_list)
            return X

In [0]:
filepath = '/content/drive/My Drive/before/sample1.dat'

In [0]:
modelpath = '/content/drive/My Drive/before/mymodel12.h5'
model = get_model()
train_model(model)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 100, 400, 3)       12        
_________________________________________________________________
mobilenetv2_0.60_100 (Model) (None, 4, 13, 1280)       958176    
_________________________________________________________________
batch_normalization_2 (Batch (None, 4, 13, 1280)       5120      
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1281      
Total params: 964,589
Trainable params: 939,847
Non-trainable params: 24,742
___________________________________________

In [0]:
model.predict(np.expand_dims(transform_image(cv2.imread('/content/drive/My Drive/before/20200505191234/20200505191234277788.jpg')), axis=0))

In [0]:
x, y = read_data(True)

In [0]:
dg = DataGenerator(x, y, 64, (100, 400), 3)

import matplotlib.pyplot as plt

for i, (x, y) in enumerate(dg):
    if(i <= 1):
        x_first = x[0]
        plt.title(y[0])
        plt.imshow(x_first)

In [0]:
for i, (x, y) in enumerate(dg):
    if(i <= 1):
        x_first = x[0]
        plt.title(y[0])
        plt.imshow(x_first)